In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [4]:
# df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
# calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
# price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# # df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [17]:
def create_dt(is_train=True, nrows= None, first_day =1200):
    prices = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv',dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min() ## 어짜피 min이 0이라 상관이 없다 
            #  encoding을 하면 -1부터 인코딩 되는데 그걸 0으로 환산한것

    cal = pd.read_csv("m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col,col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
    #         print(col,cal[col].dtypes)
            # category value들을 label encoding 하는 코드
            # 인코딩하면  -1 부터 되는데 그걸 0으로 바꿔줌
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()

    start_day = max(1 if is_train else tr_last-max_lags,first_day)
#     start_day = 1
#     print(start_day) # predict 할 때 최근 데이터만 가지고 와서 Lag를 만든다.

    numcols = [f"d_{day}" for day in range(start_day, tr_last+1)]
    print(numcols[:10])
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col:"category" for col in catcols if col != "id"})
    # 데이터를 다 불러오지 않고 필요한 데이터만 불러왔다.
    # startdate도 정해뒀다.
    dt = pd.read_csv("m5-forecasting-accuracy/sales_train_validation.csv",
                    nrows = nrows, usecols = catcols + numcols,dtype = dtype)
    print(dt.shape)
    for col in catcols : # category 컬럼 encoding
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    if not is_train:
        for day in range(tr_last+1,tr_last+28+1):
            dt[f"d_{day}"]  = np.nan

    dt = pd.melt(dt, id_vars = catcols,
                value_vars= [col for col in dt.columns if col.startswith("d_")],
                var_name ="d",
                value_name = "sales")
    print(1, dt.shape)
    dt = dt.merge(cal, on= "d", copy =False)
    print(2, dt.shape)
    dt = dt.merge(prices, on = ["store_id","item_id","wm_yr_wk"],copy = False)
    print(dt.shape)
    return dt
    # print(dt.head())




In [18]:
%%time
### 이렇게 하는게 더 빠른가?
def create_fea(dt):
    lags= [7,28]
    lag_cols = [f"lag_{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    # dt.head()
    wins = [7,28]
    for win in wins:
        for lag, lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    date_features = {

            "wday": "weekday",
            "week": "weekofyear",
            "month": "month",
            "quarter": "quarter",
            "year": "year",
            "mday": "day",
    #         "ime": "is_month_end",
    #         "ims": "is_month_start",
        }

    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else: # 새로 만드는 것도 있다.
            print(date_feat_name)
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

    

Wall time: 0 ns


In [19]:
%%time
FIRST_DAY = 350
df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

['d_350', 'd_351', 'd_352', 'd_353', 'd_354', 'd_355', 'd_356', 'd_357', 'd_358', 'd_359']
(30490, 1570)
1 (47686360, 8)
2 (47686360, 21)
(40718219, 22)
Wall time: 29.2 s


(40718219, 22)

In [20]:
df.shape

(40718219, 22)

In [21]:
df.dropna(inplace = True)
df.shape

(40718219, 22)

In [22]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [23]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

Wall time: 18.2 s


In [24]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,                                                 
    'num_iterations' : 1200,
    'num_leaves': 128,                                                                                                                                                                                                                                                                     
    "min_data_in_leaf": 100,
}

In [25]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 3.5492
[40]	valid_0's rmse: 3.22789
[60]	valid_0's rmse: 3.1107
[80]	valid_0's rmse: 3.03203
[100]	valid_0's rmse: 3.00927
[120]	valid_0's rmse: 2.93253
[140]	valid_0's rmse: 2.8968
[160]	valid_0's rmse: 2.85991
[180]	valid_0's rmse: 2.81566
[200]	valid_0's rmse: 2.78504
[220]	valid_0's rmse: 2.75908
[240]	valid_0's rmse: 2.73346
[260]	valid_0's rmse: 2.72413
[280]	valid_0's rmse: 2.69835
[300]	valid_0's rmse: 2.68376
[320]	valid_0's rmse: 2.65807
[340]	valid_0's rmse: 2.63217
[360]	valid_0's rmse: 2.61786
[380]	valid_0's rmse: 2.60247
[400]	valid_0's rmse: 2.59206
[420]	valid_0's rmse: 2.57569
[440]	valid_0's rmse: 2.56876
[460]	valid_0's rmse: 2.56159
[480]	valid_0's rmse: 2.5543
[500]	valid_0's rmse: 2.54706
[520]	valid_0's rmse: 2.54263
[540]	valid_0's rmse: 2.53767
[560]	valid_0's rmse: 2.52902
[580]	valid_0's rmse: 2.52354
[600]	valid_0's rmse: 2.51775
[620]	valid_0's rmse: 2.51411
[640]	valid_0's rmse: 2.5112
[660]	valid_0's rmse: 2.5062
[680]	valid_0's rmse

In [28]:
m_lgb.save_model("필사_model.txt")

In [34]:
import pickle

pickle.dump(m_lgb,open( "test.pkl", "wb" ))

In [27]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        # 예측된 값을 재사용을 해야하는데  나는 안썼다
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)                                                                                                                                                                                                                                                           


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_검증.csv",index=False)

['d_1856', 'd_1857', 'd_1858', 'd_1859', 'd_1860', 'd_1861', 'd_1862', 'd_1863', 'd_1864', 'd_1865']
(30490, 64)
1 (2622140, 8)
2 (2622140, 21)
(2622140, 22)
0 2016-04-25 00:00:00
week
quarter
mday
1 2016-04-26 00:00:00
week
quarter
mday
2 2016-04-27 00:00:00
week
quarter
mday
3 2016-04-28 00:00:00
week
quarter
mday
4 2016-04-29 00:00:00
week
quarter
mday
5 2016-04-30 00:00:00
week
quarter
mday
6 2016-05-01 00:00:00
week
quarter
mday
7 2016-05-02 00:00:00
week
quarter
mday
8 2016-05-03 00:00:00
week
quarter
mday
9 2016-05-04 00:00:00
week
quarter
mday
10 2016-05-05 00:00:00
week
quarter
mday
11 2016-05-06 00:00:00
week
quarter
mday
12 2016-05-07 00:00:00
week
quarter
mday
13 2016-05-08 00:00:00
week
quarter
mday
14 2016-05-09 00:00:00
week
quarter
mday
15 2016-05-10 00:00:00
week
quarter
mday
16 2016-05-11 00:00:00
week
quarter
mday
17 2016-05-12 00:00:00
week
quarter
mday
18 2016-05-13 00:00:00
week
quarter
mday
19 2016-05-14 00:00:00
week
quarter
mday
20 2016-05-15 00:00:00
week
quar

In [16]:
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.612291,0.592146,0.599873,0.602315,0.724010,0.916642,0.907589,0.697441,0.689190,...,0.779002,0.985004,0.968978,0.613898,0.593701,0.601448,0.603896,0.725911,0.931915,0.898007
1,FOODS_1_001_CA_2_validation,0.903741,0.856429,0.858531,0.891300,1.046258,1.387390,1.224174,0.876754,0.864416,...,1.029706,1.347493,1.326789,0.906114,0.858678,0.860785,0.893640,1.049005,1.371377,1.328744
2,FOODS_1_001_CA_3_validation,0.912509,0.866253,0.855478,0.854139,0.985525,1.246874,1.322142,0.947859,0.937712,...,1.003663,1.247639,1.268098,0.914905,0.868527,0.857724,0.856382,0.988112,1.233985,1.241161
3,FOODS_1_001_CA_4_validation,0.387748,0.383004,0.385477,0.385121,0.412431,0.458589,0.419123,0.417486,0.420117,...,0.438306,0.491296,0.485669,0.387748,0.383004,0.385477,0.385121,0.412431,0.450810,0.430746
4,FOODS_1_001_TX_1_validation,0.434609,0.423352,0.420779,0.420818,0.470379,0.523782,0.573856,0.470710,0.486605,...,0.517509,0.566965,0.593260,0.434609,0.423352,0.420779,0.420818,0.470379,0.544416,0.546785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.209708,0.204280,0.206620,0.220088,0.260860,0.308483,0.287629,0.209393,0.203029,...,0.258901,0.340277,0.325154,0.212425,0.206926,0.209296,0.222939,0.264239,0.324346,0.313711
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.114978,0.111543,0.113700,0.118687,0.142227,0.155008,0.147093,0.113443,0.109840,...,0.144522,0.172153,0.166225,0.115586,0.112132,0.114301,0.119314,0.142979,0.163775,0.158096
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.074731,0.076999,0.079002,0.085823,0.110197,0.129831,0.097722,0.073480,0.074562,...,0.115152,0.134162,0.112845,0.074671,0.076937,0.078938,0.085754,0.110109,0.127695,0.104315
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.058004,0.059522,0.061639,0.063770,0.078285,0.083159,0.067514,0.057000,0.057870,...,0.078696,0.089661,0.078259,0.057697,0.059207,0.061312,0.063432,0.077870,0.084330,0.070993
